# 심층 합성곱 생성적 적대 신경망 (Deep Convolutional Generative Adversarial Networks, DCGAN)

## 생성적 적대 신경망(GANs) 

- [생성적 적대 신경망](https://arxiv.org/abs/1406.2661) (Generative Adversarial Networks, GANs)은 요즘 컴퓨터 과학에서 가장 흥미로운 아이디어 중 하나  
- 두개의 모델이 적대적인 과정을 통해 동시에 훈련  
- *생성자* ("예술가")는 진짜처럼 보이는 이미지를 생성하도록 배우는 와중에, *감별자* ("예술비평가")는 가짜의 이미지로부터 진짜를 구별하게 되는 것을 배우게 됨

![생성자와 감별자를 그린 도표](https://tensorflow.org/tutorials/generative/images/gan1.png)



- 훈련과정 동안 *생성자*는 점차 실제같은 이미지를 더 잘 생성  
- *감별자*는 점차 진짜와 가짜를 더 잘 구별  
- 이 과정은 *감별자*가 가짜 이미지에서 진짜 이미지를 더이상 구별하지 못하게 될때, 평형상태에 도달

![생성자와 감별자를 그린 두번째 도표](https://tensorflow.org/tutorials/generative/images/gan2.png)

- 이 과정을 MNIST 데이터를 이용하여 구현  
- 아래의 애니메이션은 50 에포크(epoch)동안 훈련한 *생성자*가 생성해낸 연속된 이미지들을 보여줌  
- 이미지들은 랜덤한 잡음으로 부터 시작되었고, 점차 시간이 지남에 따라 손으로 쓴 숫자들을 닮아가게 됨

![출력 예시](https://tensorflow.org/images/gan/dcgan.gif)

### 데이터셋 로딩 및 준비
- 생성자와 감별자를 훈련하기위해 MNIST 데이터셋을 사용  
- 생성자는 손글씨 숫자 데이터를 닮은 숫자들을 생성할 것임

## 모델 만들기 

### 생성자

- 생성자는 시드값 (seed; 랜덤한 잡음)으로부터 CNN 의 역순으로 이미지를 생성하기 위해, `tf.keras.layers.Conv2DTranspose` (inverse Convolution 을 통한 upsampling) 층을 이용  
- stride=(2, 2) 일 경우 dimension 이 2 배로 upsampling 됨
- 처음 `Dense`층은 시드값을 인풋으로 받음  
- 그 다음 원하는 사이즈 28x28x1의 이미지가 나오도록 Conv2DTranspose 를 이용한 업샘플링을 여러번 함  
- tanh를 사용하는 마지막 층을 제외한 나머지 각 층마다 활성함수로 `tf.keras.layers.LeakyReLU`을 사용하고 있음을 주목할 것

<img src="https://camo.githubusercontent.com/45e147fc9dfcf6a8e5df2c9b985078258b9974e3/68747470733a2f2f63646e2d696d616765732d312e6d656469756d2e636f6d2f6d61782f313030302f312a33394e6e6e695f6e685044614c7539416e544c6f57772e706e67" width="700">

- "same" padding and stride = 1, 출력은 같은 크기입니다  
- "same" padding and stride = 2, 출력은 두 배 크기입니다.

(아직 훈련이 되지않은) 생성자를 이용해 이미지를 생성

model(input, training=False)   
- False - inference. No Dropout and BatchNormalization 
- True - training  

### 감별자 
- 감별자는 합성곱 신경망(Convolutional Neural Network, CNN) 기반의 이미지 분류기
- MNIST dataset 은 input_shape (28, 28, 1)  
- sigmoid output 은 probability scalar 값  
- CNN 과의 차이 : pooling layer 없고, stride 를 통하여 downsampling  

<img src="https://camo.githubusercontent.com/45e147fc9dfcf6a8e5df2c9b985078258b9974e3/68747470733a2f2f63646e2d696d616765732d312e6d656469756d2e636f6d2f6d61782f313030302f312a33394e6e6e695f6e685044614c7539416e544c6f57772e706e67" width="700">

- (아직까지 훈련이 되지 않은) 감별자를 사용하여, 생성된 이미지가 진짜인지 가짜인지 판별  
- 모델은 진짜 이미지에는 positive, 가짜 이미지에는 negative 를 출력하도록 훈련

## 손실함수와 옵티마이저 정의
- 두 모델의 손실함수와 옵티마이저를 정의  
- Discriminator 의 output 이 sigmoid 이므로, binary crossentropy 를 loss function 으로 사용 
- from_logits=True 로 지정   


- tf.keras.losses.BinaryCrossentropy()(y_true, y_pred, sample_weight=None)

### 감별자 손실함수 

- 감별자가 가짜 이미지에서 얼마나 진짜 이미지를 잘 판별하는지 수치화하는 함수  
- 진짜 이미지에 대한 감별자의 예측과 1로 이루어진 행렬을 비교하고, 가짜 (생성된) 이미지에 대한 감별자의 예측과 0으로 이루어진 행렬을 비교  
- shape 은 (256, 1) $\rightarrow$ BATCHSIZE
- real image 는 label [11111..111], fake image 는 label [00000....000] 이 ground truth  
- discriminator 는 real 은 real 로, fake 는 fake 로 바르게 판별해야 하므로 real_loss + fake_loss 가 minimize 되도록 training

### 생성자 손실함수

- 감별자를 얼마나 잘 속였는지에 대해 수치화  
- 생성자가 원활히 수행되고 있다면, 감별자는 가짜 이미지를 진짜 (또는 1)로 분류를 할 것임. 
- 생성된 이미지에 대한 감별자의 결정(fake_output)을 1로 이루어진 행렬과 비교(감별자가 감별한 결과가 모두 1 이 되어야 생성자가 감별자를 완벽히 속인 것임)  


- fake_output = discriminator(generated_images, training=True)

### 감별자와 생성자는 따로 훈련되기 때문에, 감별자와 생성자의 옵티마이저는 별도 지정.

### 체크포인트 저장

- optimizer 와 model 저장 


-  tf.train.Checkpoint(optimizer=optimizer, model=model)

## 훈련 루프 정의

- 훈련 루프는 생성자가 입력으로 랜덤시드를 받는 것으로부터 시작  
- 그 시드값을 사용하여 이미지를 생성  
- 감별자를 사용하여 (훈련 세트에서 갖고온) 진짜 이미지와 (생성자가 생성해낸) 가짜이미지를 분류  
- 각 모델의 손실을 계산하고, 그래디언트 (gradients)를 사용해 생성자와 감별자를 업데이트
<img src="adverseModel.png" width="500">

## Training

<img src="GANtraining.png" width="500">

**이미지 생성 및 저장**

-`training`이 False로 맞춰진 것을 주목. 이렇게 하면 (배치정규화를 포함하여) 모든 층들이 추론 모드로 실행

## 모델 훈련
- 위에 정의된 `train()` 메서드를 생성자와 감별자를 동시에 훈련하기 위해 호출  
- 생성적 적대 신경망을 학습하는 것은 매우 까다롭다. 생성자와 감별자가 서로를 제압하지 않는 것이 중요 (비슷한 속도로 train)
- 훈련 초반부에는 생성된 이미지는 랜덤 노이즈처럼 보이나 훈련이 진행될수록, 생성된 숫자는 점차 진짜와 비슷해짐  
- 약 50 에포크가 지난 후, MNIST 와 닮은 이미지가 생성  
- Colab에서 기본 설정으로 실행하면, 에포크마다 1분정도 소요

마지막 체크포인트를 복구합니다.

## GIF 생성